In [ ]:
# !pip install faker
# !pip install sqlalchemy

In [ ]:
from sqlalchemy import (create_engine, Column, Integer, String, Date, Numeric, Text, ForeignKey, Boolean, CheckConstraint, Enum, TIMESTAMP)
from sqlalchemy.orm import declarative_base, sessionmaker, relationship
from faker import Faker
import random
from datetime import datetime, timedelta

# Database setup
DATABASE_URL = "sqlite:///../BankDB.db"
engine = create_engine(DATABASE_URL, echo=True)
Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()
fake = Faker()

# Define ORM Models
class Customer(Base):
    __tablename__ = "customers"
    customer_id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String(100), nullable=False)
    last_name = Column(String(100), nullable=False)
    dob = Column(Date, nullable=False)
    phone_number = Column(String(15), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    address = Column(Text, nullable=False)
    created_at = Column(TIMESTAMP, default=datetime.utcnow)

class Account(Base):
    __tablename__ = "accounts"
    account_id = Column(Integer, primary_key=True, autoincrement=True)
    customer_id = Column(Integer, ForeignKey("customers.customer_id"), nullable=False)
    account_type = Column(String, CheckConstraint("account_type IN ('Savings', 'Checking', 'Loan', 'Credit Card')"), nullable=False)
    account_number = Column(String, unique=True, nullable=False)
    balance = Column(Numeric(15, 2), default=0.00)
    currency = Column(String, nullable=False)
    status = Column(String, CheckConstraint("status IN ('Active', 'Inactive', 'Closed', 'Frozen')"), default="Active")
    opened_at = Column(TIMESTAMP, default=datetime.utcnow)

class Transaction(Base):
    __tablename__ = "transactions"
    transaction_id = Column(Integer, primary_key=True, autoincrement=True)
    account_id = Column(Integer, ForeignKey("accounts.account_id"), nullable=False)
    transaction_type = Column(String, CheckConstraint("transaction_type IN ('Deposit', 'Withdrawal', 'Transfer', 'Payment')"), nullable=False)
    amount = Column(Numeric(15, 2), nullable=False)
    currency = Column(String, nullable=False)
    transaction_date = Column(TIMESTAMP, default=datetime.utcnow)
    status = Column(String, CheckConstraint("status IN ('Pending', 'Completed', 'Failed', 'Reversed')"), default="Completed")
    description = Column(Text, nullable=True)

class User(Base):
    __tablename__ = "users"
    user_id = Column(Integer, primary_key=True, autoincrement=True)
    username = Column(String, unique=True, nullable=False)
    password_hash = Column(String, nullable=False)
    role = Column(String, CheckConstraint("role IN ('Admin', 'Teller', 'CustomerService')"), nullable=False)
    created_at = Column(TIMESTAMP, default=datetime.utcnow)

# Create tables
Base.metadata.create_all(engine)

def create_fake_customers(n=50):
    customers = []
    for _ in range(n):
        customer = Customer(
            first_name=fake.first_name(),
            last_name=fake.last_name(),
            dob=fake.date_of_birth(minimum_age=18, maximum_age=80),
            phone_number=fake.unique.phone_number(),
            email=fake.unique.email(),
            address=fake.address(),
            created_at=fake.date_time_this_decade()
        )
        customers.append(customer)
    session.bulk_save_objects(customers)
    session.commit()  # Ensure data is saved before querying
    print(f"Inserted {n} customers!")


def create_fake_accounts(n=100):
    customers = session.query(Customer).all()
    if not customers:  
        print("No customers found! Make sure customer data is inserted first.")
        return

    accounts = []
    for _ in range(n):
        account = Account(
            customer_id=random.choice(customers).customer_id,
            account_type=random.choice(["Savings", "Checking", "Loan", "Credit Card"]),
            account_number=fake.unique.bban(),
            balance=0.00,  # Start with zero balance
            currency="INR",
            status=random.choice(["Active", "Inactive", "Closed", "Frozen"]),
            opened_at=fake.date_time_this_decade()
        )
        accounts.append(account)

    session.bulk_save_objects(accounts)
    session.commit()
    print(f"Inserted {n} accounts!")


def create_fake_transactions(n=300):
    accounts = session.query(Account).all()
    transactions = []

    account_last_transaction_date = {}  # Dictionary to track last transaction date per account

    for account in accounts:
        # Start transaction dates from account creation date
        account_last_transaction_date[account.account_id] = account.opened_at

    for _ in range(n):
        account = random.choice(accounts)
        last_transaction_date = account_last_transaction_date[account.account_id]

        # Generate the next transaction date ensuring it's after the last one
        transaction_date = last_transaction_date + timedelta(days=random.randint(1, 30))

        transaction = Transaction(
            account_id=account.account_id,
            transaction_type=random.choice(["Deposit", "Withdrawal", "Transfer", "Payment"]),
            amount=round(random.uniform(10.00, 10000.00), 2),
            currency=random.choice(["INR"]),
            status=random.choice(["Pending", "Completed", "Failed", "Reversed"]),
            transaction_date=transaction_date,
            description=fake.sentence()
        )
        transactions.append(transaction)

        # Update the last transaction date for the account
        account_last_transaction_date[account.account_id] = transaction_date

    session.bulk_save_objects(transactions)
    session.commit()
    print(f"Inserted {n} transactions with ordered dates!")


def create_fake_users(n=10):
    roles = ["Admin", "Teller", "CustomerService"]
    users = []
    for _ in range(n):
        user = User(
            username=fake.user_name(),
            password_hash=fake.sha256(),
            role=random.choice(roles),
            created_at=fake.date_time_this_decade()
        )
        users.append(user)
    session.bulk_save_objects(users)
    session.commit()


In [ ]:
# Populate database
create_fake_customers(500)
create_fake_accounts(250)
create_fake_transactions(10000)
create_fake_users(10)

print("Database successfully populated with fake data!")

In [ ]:
from sqlalchemy.schema import CreateTable
from sqlalchemy.dialects import sqlite
# from your_module import Customer  # Import your ORM model

# Generate the SQL statement
sql_statement = str(CreateTable(Account.__table__).compile(dialect=sqlite.dialect()))

print(sql_statement)
